# ECA Dynamic Examination

In [1]:
import numpy as np

def bin_from_int(x):
    
    wolfram_code = [int(bit) for bit in format(x, f'08b')]
    return wolfram_code

config = 
lut = np.array([1, 0, 0, 0, 1, 0, 0])

def eca_evolve(lut, x):
    

ModuleNotFoundError: No module named 'eca'

In [91]:
import numpy as np

def bin_from_int(x):
    return np.array([int(bit) for bit in format(x, f'08b')])
    #return wolfram_code

def eca_get_lut(rule_num):
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(8)])

def evolve_eca(rule, configuration):
    conf = np.stack([np.roll(configuration, 1), configuration, np.roll(configuration, -1)])
    index = 7 - (conf[0] * 4 + conf[1] * 2 + conf[2])
    return rule[index]


In [93]:
initial_config = np.array([1, 0, 0, 0, 1, 0, 0])
x = bin_from_int(30)

print(x)
evolve_eca(x, initial_config)



[0 0 0 1 1 1 1 0]


array([1, 1, 0, 1, 1, 1, 1])

In [24]:
from itertools import repeat
nums = [i for i in range(256)]

for num in repeat(nums, 2):
    print(num)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [85]:
from time import time

start = time()

for num in repeat(nums, 1000):
    for i in num:
        bin_from_int(i)

stop = time()
print(stop-start)


0.5592849254608154


In [86]:
from time import time

start = time()

for num in repeat(nums, 1000):
    for i in num:
        eca_get_lut(i)

stop = time()
print(stop-start)

0.5258417129516602


In [98]:
from src.eca import ECA

#automaton = ECA.init_random(64)
automaton = ECA.init_config([1, 0, 0, 0, 1, 0, 0])
automaton.evolve(30, 64)
automaton.visualize()


ModuleNotFoundError: No module named 'src'